### Example of ED ——  Transverse-field Ising model on a N-site ring

$$
\begin{gathered}
H=-J \sum_{i=1}^N\left(g \sigma_i^x+\sigma_i^z \sigma_{i+1}^z\right) \quad J, g \geq 0 \\
\sigma^x=\left(\begin{array}{ll}
0 & 1 \\
1 & 0
\end{array}\right), \sigma^z=\left(\begin{array}{cc}
1 & 0 \\
0 & -1
\end{array}\right)
\end{gathered}
$$

In [ ]:
import numpy as np
import scipy.linalg as LA
from scipy.sparse import coo_matrix
import pandas as pd
import math
import matplotlib.pyplot as plt
from itertools import combinations

The translational operator $T$ is performed using RotLBit(i, L, n).

In [ ]:
# Binary Encoding for the Base Set

# Calculate the number of '1' in the base set
def CountBit(i):
    return np.binary_repr(i).count('1')

# Flip the nth bit of a binary number
def FilpBit(i, n):
    return i ^ (1 << n)

# Read the nth bit of a binary number
def ReadBit(i, n)->int:
    return (i & (1 << n)) >> n

# Read the value of an n-bit binary number starting from the kth bit
def PickBit(i, k, n):
    return (i & ((2 ** n -1) << k)) >> k

# Left shift by n bits
def RotLBit(i, L, n):
    return (PickBit(i, 0, L - n) << n) + (i >> (L - n))

# Right shift by n bits
def RotRBit(i, L, n):
    return (PickBit(i, 0, n) << (L - n)) + (i >> n)


ActingH(n) calculates the result of Hamiltonian acting on a state $|r\rangle$ in transverse-field Ising model.

In [ ]:
def ActingH(n, L, J, g):
    ising = [((i), (i + 1) % L) for i in range(0, L)]
    output = []
    
    #Ising Bond
    for i in range(0, L):
        m = n ^ (1 << i)
        output.append((m, - J * g))
    
    diag = 0

    #Transverse Field
    for j in ising:
        if(ReadBit(n, j[0]) == ReadBit(n, j[1])):
            diag = diag - J
        else:
            diag = diag + J
    
    output.append((n, diag))
    
    return output

Translational symmetry should be taken into consideration when constructing the Hamiltonian matirx. For a given N-site model, we only need to calculate the Hamiltonian matirx $H_k$ for each k sector with k ranging from $0$ to $N-1$.
$$H = H_{0} \oplus H_{2} \oplus H_{k} \oplus \cdots \oplus H_{N - 1}$$

When constructing the basis, we need to find the representative state(RS) and determine wheter it belong to a specific $k$ sector using the rule

$$kR = \text{integer} \cdot N$$

where $R$ means aftering translating $R$ steps, the orginal state is obtained $T^R|r_k\rangle = |r_k\rangle$.

The matrix element between two state $|r_k\rangle$ and $|r_k^\prime\rangle$ is given by
$$
\left\langle r_k^{\prime}|H| r_k\right\rangle=\frac{\left\langle r^{\prime}\left|P_k^{\dagger} H P_k\right| r\right\rangle}{\sqrt{\left\langle r^{\prime}\left|P_k\right| r^{\prime}\right\rangle\left\langle r\left|P_k\right| r\right\rangle}}=\frac{\left\langle r^{\prime}|P_k H |r\right\rangle}{\sqrt{\left\langle r^{\prime}\left|P_k\right| r^{\prime}\right\rangle\left\langle r\left|P_k\right| r\right\rangle}}
$$




In [ ]:
# Building of all N-site basis
def BuildBasisN(L):
    basisN = []

    for i in range(0, 2 ** L):
            basisN.append(i)

    return basisN

# Building of basis for each fixed-momentum Hilbert space, only RS is necessary to be stored
def BuildBasisNk(L, k):
    basisNk = []
    basisN = BuildBasisN(L)

    for n in basisN:
        #find the RS
        Tlist = [RotLBit(n, L, i) for i in range(0, L + 1)]
        m = min(Tlist)
        R = Tlist[1:].index(n) + 1
        #kR = integer * N for spin systems
        if(m == n and (k * R) % L == 0):
            basisNk.append(n)
    
    return basisNk

# Building of Hamiltonian matrix in the form of sparse matrix
def BuildHNk(L, k, J, g):
    HNk_col = []
    HNk_row = []
    HNk_val = []
    pos = {}
    basisNk = BuildBasisNk(L, k)
    for n in basisNk:
        b = basisNk.index(n)
        output = ActingH(n, L, J, g)
        for (m, h) in output:
            Tlist = [RotLBit(m, L, i) for i in range(0, L + 1)]
            #Find the RS of m and calculate the distance beteen them
            m_rs = min(Tlist)
            Rm = Tlist[1:].index(m) + 1
            d = Tlist.index(m_rs)

            Tlist = [RotLBit(n, L, i) for i in range(0, L + 1)]
            Rn = Tlist[1:].index(n) + 1
            # After acting the Hamiltonian, the result may contain non-representative state, 
            # so the matrix elements is a sum of all states belonging to the same RS
            if(m_rs in basisNk):
                a = basisNk.index(m_rs)
                if(not (a, b) in pos):
                    HNk_col.append(a)
                    HNk_row.append(b)
                    # Normalization coefficient is considered
                    HNk_val.append(h * np.exp(1j * k * d * 2 * math.pi / L) * np.sqrt(Rn) / np.sqrt(Rm))
                    pos[(a, b)] = len(HNk_col) - 1
                else:
                    HNk_val[pos[(a, b)]] = HNk_val[pos[(a, b)]] + h * np.exp(1j * k * d * 2 * math.pi / L) * np.sqrt(Rn) / np.sqrt(Rm)
    
    return HNk_col, HNk_row, HNk_val, len(basisNk), basisNk
    

In [ ]:
#Display the result
def Task(L):
    J_list = [1]
    g_list = [1]

    E, V = [], []

    for J, g in zip(J_list, g_list):
        count = 0
        Hx = []
        Hy = []
        Hn = []

        for k in range(0, L):
            X, Y, N, C, basis= BuildHNk(L, k, J, g)
            Hx = Hx + [p + count for p in X]
            Hy = Hy + [p + count for p in Y]
            Hn = Hn + [p for p in N]
            count = count + C

        Ham = (coo_matrix((Hn, (Hx, Hy)),shape=(count, count)).tocsc())
        E, V = LA.eig(Ham.toarray()) 
        
    return E, V

Task(4)



### Reference
1. Jung, J. H., & Noh, J. D. (2020). Guide to Exact Diagonalization Study of Quantum Thermalization. Journal of the Korean Physical Society, 76(8), 670-683.

